In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import os
import datetime
from sklearn.preprocessing import StandardScaler

%load_ext tensorboard

In [2]:
def data_preprocessing(dataframe, is_train):
    # Drop the following columns
    df_dropped = dataframe.drop(['Name', 'Cabin', 'PassengerId', 'Ticket'], axis=1)
    
    # 2 are missing from training but non on test set so
    
    if is_train==True:
        df_dropped["Embarked"] = df_dropped["Embarked"].fillna(value="C")
        
        # Also for some special ages
        
        mean_value = df_dropped[(df_dropped["Survived"] == 0) & (df_dropped["Pclass"] == 3) & (df_dropped["Parch"] == 2) ].Age.mean()
        
        df_dropped = df_dropped.assign(Age=np.where(df_dropped.Survived.eq(0) & df_dropped.Pclass.eq(3) & df_dropped.SibSp.eq(8)& df_dropped.Age.isnull(), mean_value, df_dropped.Age))
        
        # Fill data with mean
        l_col = ['Survived','Pclass','Sex','Embarked','SibSp','Parch']
        df_dropped['Age'] = df_dropped['Age'].fillna(df_dropped.groupby(l_col)['Age'].transform('mean'))
        

    if is_train==False:
                
        # Fill data with mean
        l_col = ['Pclass','Sex','Embarked','SibSp','Parch']
        df_dropped['Age'] = df_dropped['Age'].fillna(df_dropped.groupby(l_col)['Age'].transform('mean'))
        
        
    # Drop the remaing null ages column
    
    cleaned_df = df_dropped.dropna(subset=['Age'])
    
    # Hot en-code male and female
    cleaned_df['Sex'] = cleaned_df['Sex'].map({'female': 1, 'male': 0}).astype(int)

    # Hot en-code Embarked
    enbarked_one_hot = pd.get_dummies(cleaned_df['Embarked'], prefix='Embarked')
    cleaned_df = cleaned_df.drop('Embarked', axis=1)
    cleaned_df = cleaned_df.join(enbarked_one_hot)
    
    x = np.asarray(cleaned_df.drop('Survived', axis=1))
    scale = StandardScaler()
    x_values = scale.fit_transform(x)

    y_label = cleaned_df["Survived"].values
    
    return x_values, y_label

In [3]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

x_train, y_train=data_preprocessing(df_train, is_train=True)
#x_test, y_test=data_preprocessing(df_test, is_train=False)

C:\ProgramData\anaconda3\envs\tf_21\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model = Sequential([
    Dense(9, activation="relu", kernel_initializer='glorot_normal',input_dim=x_train.shape[1]),
    Dense(7, activation="relu"),
    Dense(5, activation="relu"),
    Dense(3, activation="relu"),
    Dense(1, activation="sigmoid")

])

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.fit(x_train, 
          y_train, 
          epochs=1000,
          callbacks=[tensorboard_callback])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 875 samples
Epoch 1/1000
875/875 [==============================] - 0s 382us/sample - loss: 0.6454 - acc: 0.6183
Epoch 2/1000
875/875 [==============================] - 0s 72us/sample - loss: 0.6144 - acc: 0.6183
Epoch 3/1000
875/875 [==============================] - 0s 72us/sample - loss: 0.5904 - acc: 0.6183
Epoch 4/1000
875/875 [==============================] - 0s 67us/sample - loss: 0.5746 - acc: 0.6149
Epoch 5/1000
875/875 [==============================] - 0s 66us/sample - loss: 0.5639 - acc: 0.6469
Epoch 6/1000
875/875 [==============================] - 0s 65us/sample - loss: 0.5562 - acc: 0.7246
Epoch 7/1000
875/875 [==============================] - 0s 68us/sample - loss: 0.5496 - acc: 0.7463
Epoch 8/1000
875/875 [==============================] - 0s 72us/sample - loss: 0.5438 - acc: 0.7726
Epo

In [6]:
model.evaluate(x_train, y_train)[1]

875/875 [==============================] - 0s 46us/sample - loss: 0.2873 - acc: 0.8949


0.89485717